Импортируем все необходимые библиотеки

In [82]:
pip install natasha scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [280]:
import string, re
from natasha import Doc, Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pymorphy2
from collections import defaultdict

1. Лемматизация и стемминг текста

Эта функция выполняет лемматизацию и стемминг для русского текста с помощью библиотеки natasha
1) Текст разбивается на предложения и слова
2)  Слова преобразуются в нормальные формы
3)  Загружаются предобученные векторные представления слов
4)  Анализируется морфология
После создается объект и разбивается на токены, добавляется морфологический анализ
Затем прорабатывается цикл поиска лемм и добавления их в список
В стемменге усекаются слова до основы

In [341]:
def lemm_and_stem(text):
    segmenter = Segmenter()
    Vocab = MorphVocab()
    Embeddings = NewsEmbedding()
    Tagger = NewsMorphTagger(Embeddings)
    Stemmer = SnowballStemmer("russian")
    morph = pymorphy2.MorphAnalyzer()
    
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(Tagger)
    
    lems = []
    for token in doc.tokens:
        if token.lemma:  
            lems.append(token.lemma)
        else:
            parsed = morph.parse(token.text)[0]
            lems.append(parsed.normal_form)
    
    stems = [Stemmer.stem(t.text) for t in doc.tokens]
    
    return lems, stems

2. Функция для токенизации всех символов из ASCII
    1) Все символы, не являющиеся частью ASCII, будут проигнорированы.
    2) Текст будет разбит на токены.


In [343]:
def tokenize(text):
    if isinstance(text, str):
        token = [char.lower() for char in text]
    elif isinstance(text, list):
        token = [char.lower() for word in text for char in word]

    return token

3. Функция для векторизации всех символов из ASCII
    1) Векторизация текста с помощью CountVectorizer из sklearn.
    2) Все символы, не являющиеся частью ASCII, будут проигнорированы.

In [346]:
def vectorize(token):
    char_freq = defaultdict(int)
    for char in token:
        char_freq[char] += 1

    char_to_index = {char: idx for idx, char in enumerate(char_freq.keys())}

    vectorized_output = np.zeros(len(char_freq), dtype=int)
    for char in token:
        vectorized_output[char_to_index[char]] += 1

    return ' '.join(map(str, vectorized_output))

4. Токенизация и векторизация текста после лемматизации и стемминга
    1) Лемматизация и стемминг текста.
    2) Токенизация ASCII символов.
    3) Векторизация текста с использованием CountVectorizer.

Русскоязычный текст(лемматизированный и стеммированный)

In [352]:
text = "По вечерам над ресторанами Горячий воздух дик и глух, И правит окриками пьяными Весенний и тлетворный дух."

lemm_text, stem_text = lemm_and_stem(text)
print("Лемматизированный текст:", lemm_text, '\n', '\nCтеммированный текст:', stem_text)
 
tokens = tokenize(text)
print("\nТокены ASCII:", tokens)

vectorized = vectorize(text)
print("\nВекторизация ASCII:", vectorized)

tokenize_lemm = tokenize(lemm_text)
print("\nТокенизация после лемматизации:", tokenize_lemm)
vectors = vectorize(tokenize_lemm)
print("\nВекторизация после лемматизации:", vectors)

Лемматизированный текст: ['по', 'вечер', 'над', 'ресторан', 'горячий', 'воздух', 'дик', 'и', 'глухой', ',', 'и', 'править', 'окрик', 'пьяный', 'весенний', 'и', 'тлетворный', 'дух', '.'] 
 
Cтеммированный текст: ['по', 'вечер', 'над', 'ресторан', 'горяч', 'воздух', 'дик', 'и', 'глух', ',', 'и', 'прав', 'окрик', 'пьян', 'весен', 'и', 'тлетворн', 'дух', '.']

Токены ASCII: ['п', 'о', ' ', 'в', 'е', 'ч', 'е', 'р', 'а', 'м', ' ', 'н', 'а', 'д', ' ', 'р', 'е', 'с', 'т', 'о', 'р', 'а', 'н', 'а', 'м', 'и', ' ', 'г', 'о', 'р', 'я', 'ч', 'и', 'й', ' ', 'в', 'о', 'з', 'д', 'у', 'х', ' ', 'д', 'и', 'к', ' ', 'и', ' ', 'г', 'л', 'у', 'х', ',', ' ', 'и', ' ', 'п', 'р', 'а', 'в', 'и', 'т', ' ', 'о', 'к', 'р', 'и', 'к', 'а', 'м', 'и', ' ', 'п', 'ь', 'я', 'н', 'ы', 'м', 'и', ' ', 'в', 'е', 'с', 'е', 'н', 'н', 'и', 'й', ' ', 'и', ' ', 'т', 'л', 'е', 'т', 'в', 'о', 'р', 'н', 'ы', 'й', ' ', 'д', 'у', 'х', '.']

Векторизация ASCII: 1 6 16 4 6 2 7 6 4 6 4 2 4 10 1 2 3 1 3 3 3 1 2 1 1 2 1 2 1 1

Токенизация 